<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [110]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [111]:
import folium
import pandas as pd

In [112]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


In [113]:
## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [114]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [115]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [116]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [117]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [141]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for index, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    name = row['Launch Site']
    
    # Add a circle around the launch site
    folium.Circle(
        location=coordinate,
        radius=10000,  # Radius in meters
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.2
    ).add_to(site_map)

    # Add a marker for the launch site
    folium.Marker(
        location=coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{name}</b></div>'
        )
    ).add_to(site_map)

site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [142]:
# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [143]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [144]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [145]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

def get_marker_color(launch_class):
    return 'green' if launch_class == 1 else 'red'

spacex_df['marker_color']= spacex_df['class'].apply(get_marker_color)

*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [146]:
for index, record in spacex_df.iterrows():
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        popup=record['Launch Site'],
        icon=folium.Icon(color =record['marker_color'])
    )
    marker_cluster.add_child(marker)

# Add marker_cluster to the site_map
site_map.add_child(marker_cluster)

# Save the map to an HTML file
site_map.save("launch_sites_map_with_clusters.html")

# Display the map in the notebook (if running in a Jupyter environment)
site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [147]:
# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [148]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [149]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0
    if lat1 and lat2 and lon1 and lon2 is not None:
        lat1 = radians(lat1)
        lon1 = radians(lon1)
        lat2 = radians(lat2)
        lon2 = radians(lon2)

        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        
        distance = R * c
        return distance
    else :
        return None

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [170]:
data={
    'railway': [(28.5623, -80.57733), (28.5632, -80.57682), (28.57325,-80.65398),(34.63349,-120.62546) ],
    'highway': [(28.56247,-80.57069),(28.56351,-80.57083),(28.57724,-80.64724),(None)],
    'city': [(28.3502,-80.699302),(28.3502,-80.699302),(28.59644,-80.8408),(34.6241,-120.45848)],
    'coastline': [(28.56216,-80.56772),(28.56335,-80.56795),(28.57454,-80.71754),(34.63349,-120.62664)]
}

data_df= pd.DataFrame(data)
data_df.head()

data_df['Launch Site']= launch_sites_df['Launch Site']
data_df['coords']= launch_sites_df.apply(lambda row: (row['Lat'], row['Long']), axis = 1)
proximity_df = data_df[['Launch Site', 'coords', 'railway', 'highway', 'city', 'coastline']]
proximity_df.head()

,Launch Site,coords,railway,highway,city,coastline
0,CCAFS LC-40,"(28.56230197, -80.57735648)","(28.5623, -80.57733)","(28.56247, -80.57069)","(28.3502, -80.699302)","(28.56216, -80.56772)"
1,CCAFS SLC-40,"(28.56319718, -80.57682003)","(28.5632, -80.57682)","(28.56351, -80.57083)","(28.3502, -80.699302)","(28.56335, -80.56795)"
2,KSC LC-39A,"(28.57325457, -80.64689529)","(28.57325, -80.65398)","(28.57724, -80.64724)","(28.59644, -80.8408)","(28.57454, -80.71754)"
3,VAFB SLC-4E,"(34.63283416, -120.6107455)","(34.63349, -120.62546)",None,"(34.6241, -120.45848)","(34.63349, -120.62664)"


In [171]:
distances_list = []

# Calculate distances
for _, row in proximity_df.iterrows():
    site = row["Launch Site"]
    coords = row["coords"]
    distances = {"Launch Site": site}
    for proximity in ["railway", "highway", "city", "coastline"]:
        if row[proximity] is not None:
            distances[proximity + " distance"] = calculate_distance(coords[0], coords[1], row[proximity][0], row[proximity][1])
        else:
            distances[proximity + " distance"] = None
    distances_list.append(distances)

# Create the distance_df DataFrame using pd.concat
distance_df = pd.concat([pd.DataFrame([dist]) for dist in distances_list], ignore_index=True)

# Display the resulting DataFrame
distance_df.head()

,Launch Site,railway distance,highway distance,city distance,coastline distance
0,CCAFS LC-40,0.002596,0.651535,26.434742,0.941549
1,CCAFS SLC-40,0.000314,0.586211,26.547251,0.866698
2,KSC LC-39A,0.692054,0.444576,19.113832,6.902208
3,VAFB SLC-4E,1.348656,None,13.970061,1.456505


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [172]:
average_lat = proximity_df['coords'].apply(lambda x: x[0]).mean()
average_lon = proximity_df['coords'].apply(lambda x: x[1]).mean()
m = folium.Map(location=[average_lat, average_lon], zoom_start=5)

# Create a MarkerCluster
marker_cluster = MarkerCluster()

# Add markers for each launch site in proximity_df
for index, record in spacex_df.iterrows():
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        popup=record['Launch Site'],
        icon=folium.Icon(color =record['marker_color'])
    )
    marker_cluster.add_child(marker)

# Add marker_cluster to the site_map
m.add_child(marker_cluster)

# Function to add polylines
def add_polyline(launch_site, launch_coords, coastline_coords, distance):
    folium.PolyLine(
        locations=[launch_coords, coastline_coords],
        color='red',
        weight=2.5,
        opacity=1
    ).add_to(m)

    folium.Marker(
        location=coastline_coords,
        icon=folium.DivIcon(
            html=f'<div style="font-size: 12pt; color: red;">{distance} KM</div>'
        )
    ).add_to(m)

# Iterate over each launch site in the DataFrame
for _, row in proximity_df.iterrows():
    launch_site = row["Launch Site"]
    launch_coords = row["coords"]
    coastline_coords = row["coastline"]
    distance = distance_df[distance_df["Launch Site"] == launch_site]["coastline distance"].values[0]
    if coastline_coords is not None:
        add_polyline(launch_site, launch_coords, coastline_coords, distance)


# Save the map to an HTML file
m.save('launch_sites_to_coastlines.html')

# Display the map in the Jupyter Notebook (if you're using one)
m

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [186]:
average_lat = proximity_df['coords'].apply(lambda x: x[0]).mean()
average_lon = proximity_df['coords'].apply(lambda x: x[1]).mean()
n = folium.Map(location=[average_lat, average_lon], zoom_start=5)

# Function to add markers and polylines
def add_polyline(launch_site, launch_coords, target_coords, distance, label, color):
    folium.Marker(
        location=target_coords,
        popup=f'{label}: {distance} KM',
        icon=folium.Icon(color=color, icon='')
    ).add_to(n)
    # Draw a PolyLine between the launch site and the target point
    folium.PolyLine(
        locations=[launch_coords, target_coords],
        color=color,
        weight=2.5,
        opacity=1
    ).add_to(n)

# Iterate over each launch site in the DataFrame
for idx, row in proximity_df.iterrows():
    launch_site = row["Launch Site"]
    launch_coords = row["coords"]
    color = colors[idx % len(colors)]
    
    # Prepare the popup content with proximity distances
    popup_content = f'<b>{launch_site}</b><br>'
    for proximity, label in zip(["railway", "highway", "city", "coastline"],
                                ["Railway", "Highway", "City", "Coastline"]):
        distance = distance_df[distance_df["Launch Site"] == launch_site][f"{proximity} distance"].values[0]
        if pd.notna(distance):
            popup_content += f'{label}: {distance} KM<br>'
    
    # Add a marker for the launch site with the popup content
    folium.Marker(
        location=launch_coords,
        popup=folium.Popup(popup_content, max_width=300),
        icon=folium.Icon(color= color, icon='info-sign')
    ).add_to(n)
    
    # Add markers and polylines for each proximity point
    for proximity, label in zip(["railway", "highway", "city", "coastline"],
                                ["Railway", "Highway", "City", "Coastline"]):
        target_coords = row[proximity]
        if target_coords is not None:
            add_polyline(launch_site, launch_coords, target_coords, distance, label, color)

# Save the map to an HTML file
n.save('launch_sites_to_proximities.html')

# Display the map in the Jupyter Notebook (if you're using one)
n

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2022-01-01)


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
